In [1]:
import os, sys, logging, importlib, math, time
import rasterio, affine

import geopandas as gpd
import pandas as pd

from zipfile import ZipFile
from affine import Affine
from rasterio import features
from rasterio.mask import mask
from rasterio.features import rasterize

import GEP
from GEP import gep_results

'''prints the time along with the message'''
def tPrint(s):
    print("%s\t%s" % (time.strftime("%H:%M:%S"), s))

# Mapping electrification
Code to download results from S3

```
aws s3 cp s3://wbg-geography01/GEP/models-august-2019/extracted_scenario_1_1_0_1_1_0/ /media/gost/DATA1/GEP/Scenario_1_1_0_1_1_0/ --recursive
```

We need to convert certain fields in the results to a raster; these fields need to be calculated from the existing data

1. Population already electrified 2030
    a. Rasterize field **ElecPop**
2. New population electrified by grid 2030
    a. Filter data where **FinalElecCode2030** == 1.0
    b. Set **NewConnections2025** = 0 where **FinalElecCode2025** == 99.0
    c. Combine **NewConnections2030** with **NewConnections2025**
3. New population electrified by off grid 2030
    a. Filter data where **FinalElecCode2030** != 1.0
    b. Set **NewConnections2025** = 0 where **FinalElecCode2025** != **FinalElecCode2030**
    c. Combine **NewConnections2030** with **NewConnections2025**

In [37]:
# delete all existing rasterized results
'''all_rasters = []
for root, dirs, files in os.walk('/media/gost/DATA1/GEP/Clusters'):
    for f in files:
        if "rasterized_gep" in f:
            all_rasters.append(os.path.join(root, f))
for f in all_rasters:
    os.remove(f)'''

In [48]:
resolution = 0.01
elecRate = 0.90
year = 2030
cluster_folder = '/media/gost/DATA1/GEP/Clusters'
scenario_base = '/media/gost/DATA1/GEP/Scenarios'
vrt_folder = '/media/gost/DATA1/GEP/GEP_VRTs'


In [49]:
import GEP
importlib.reload(GEP)
from GEP import gep_results
for scenario in ['1_1_1_1_1_0',"1_1_0_1_1_0","0_1_0_0_0_0","0_0_0_0_0_0"]:
    scenario_folder = os.path.join(scenario_base, f'Scenario_{scenario}')
    # Identify countries to process
    zipFiles = []
    processedCountries = []
    for root, folders, files in os.walk(scenario_folder):    
        processedCountries.append([f for f in folders])
        for f in files:
            if f[-4:] == ".csv":
                zipFiles.append(f[:-4])
    processedCountries = [item[:4] for sublist in processedCountries for item in sublist]
    # Loop through all countries and rasterize
    for country in processedCountries:
        tPrint(f"Processing {country} {scenario}")
        xx_so = gep_results(country, 
                    scenariosFolder=scenario_folder,
                    clustersFolder = cluster_folder,
                    scenario=scenario)
        outFile = os.path.join(xx_so.countryClustersFolder, "rasterized_gep_%s_%s_%s_gt10.tif" % (resolution, scenario, year))    
        if not os.path.exists(outFile):
            res = xx_so.join_results()
            res = res.loc[res['Pop2030'] > 10,]
            # Identify already electrified settlements and set value to 98
            res['already_electrified'] = (res['ElecPop'] / res['Pop']) > elecRate
            res.loc[res['already_electrified'] == 1, 'FinalElecCode%s' % year] = 98
            xx_so.rasterize_results(res, outFile, res=resolution, field='FinalElecCode%s' % year)
        else:
            print(f"{outFile} already exists")
    
    ### Run a series of GDAL commands to create No Data rasters and stack them
    with open(os.path.join(vrt_folder, f'create_vrt_{resolution}_{scenario}_{year}.txt'), 'w') as vrt_file:
        # Convert 0 in data to no data
        outFolder = cluster_folder
        allImages = []
        for root, folders, files in os.walk(outFolder):
            for f in files:
                if "%s_%s_%s_gt10.tif" % (resolution, scenario, year) in f:
                    allImages.append(os.path.join(root, f))

        for inFile in allImages:
            outFile = inFile.replace(".tif", "_noData.tif")
            vrt_file.write(f'gdal_translate -of GTiff -a_nodata 0 {inFile} {outFile}')
            vrt_file.write("\n")
        vrt_file.write(f'find {cluster_folder} -name "*{resolution}_{scenario}_{year}_gt10_noData.tif" > tifFiles.txt')
        vrt_file.write("\n")
        vrt_file.write(f"gdalbuildvrt {vrt_folder}/combined_gep_rasterized_{resolution}_{scenario}_{year}.vrt -input_file_list tifFiles.txt")
        vrt_file.write("\n")

14:11:24	Processing gm-1 1_1_1_1_1_0
14:11:26	Processing mw-1 1_1_1_1_1_0
14:12:07	Processing ao-1 1_1_1_1_1_0
14:12:47	Processing bd-1 1_1_1_1_1_0
14:13:53	Processing bf-1 1_1_1_1_1_0
14:14:54	Processing bi-1 1_1_1_1_1_0
14:15:18	Processing bj-1 1_1_1_1_1_0
14:15:39	Processing bw-1 1_1_1_1_1_0
14:15:47	Processing cd-1 1_1_1_1_1_0
14:17:00	Processing cf-1 1_1_1_1_1_0
14:17:09	Processing cg-1 1_1_1_1_1_0
14:17:12	Processing ci-1 1_1_1_1_1_0
14:17:41	Processing cm-1 1_1_1_1_1_0
14:18:14	Processing dj-1 1_1_1_1_1_0
14:18:15	Processing er-1 1_1_1_1_1_0
14:18:22	Processing et-1 1_1_1_1_1_0
14:22:34	Processing ga-1 1_1_1_1_1_0
14:22:38	Processing gh-1 1_1_1_1_1_0
14:23:08	Processing gn-1 1_1_1_1_1_0
14:23:26	Processing gq-1 1_1_1_1_1_0
14:23:27	Processing gw-1 1_1_1_1_1_0
14:23:29	Processing hn-1 1_1_1_1_1_0
14:23:50	Processing ht-1 1_1_1_1_1_0
14:24:04	Processing ke-1 1_1_1_1_1_0
14:25:33	Processing kh-1 1_1_1_1_1_0
14:26:07	Processing km-1 1_1_1_1_1_0
14:26:08	Processing lr-1 1_1_1_1_1_0
1

/home/gost/anaconda3/envs/Geog/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


14:48:19	Processing zm-1 1_1_1_1_1_0
14:50:11	Processing zw-1 1_1_1_1_1_0
14:51:40	Processing gm-1 1_1_0_1_1_0
14:51:42	Processing mw-1 1_1_0_1_1_0
14:52:24	Processing ao-1 1_1_0_1_1_0
14:53:04	Processing bd-1 1_1_0_1_1_0
14:54:12	Processing bf-1 1_1_0_1_1_0
14:55:13	Processing bi-1 1_1_0_1_1_0
14:55:37	Processing bj-1 1_1_0_1_1_0
14:55:58	Processing bw-1 1_1_0_1_1_0
14:56:06	Processing cd-1 1_1_0_1_1_0
14:57:16	Processing cf-1 1_1_0_1_1_0
14:57:25	Processing cg-1 1_1_0_1_1_0
14:57:29	Processing ci-1 1_1_0_1_1_0
14:57:56	Processing cm-1 1_1_0_1_1_0
14:58:28	Processing dj-1 1_1_0_1_1_0
14:58:28	Processing er-1 1_1_0_1_1_0
14:58:35	Processing et-1 1_1_0_1_1_0
15:02:43	Processing ga-1 1_1_0_1_1_0
15:02:46	Processing gh-1 1_1_0_1_1_0
15:03:15	Processing gn-1 1_1_0_1_1_0
15:03:32	Processing gq-1 1_1_0_1_1_0
15:03:33	Processing gw-1 1_1_0_1_1_0
15:03:36	Processing hn-1 1_1_0_1_1_0
15:03:55	Processing ht-1 1_1_0_1_1_0
15:04:09	Processing ke-1 1_1_0_1_1_0
15:05:36	Processing kh-1 1_1_0_1_1_0
1

/home/gost/anaconda3/envs/Geog/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


15:27:43	Processing zm-1 1_1_0_1_1_0
15:29:32	Processing zw-1 1_1_0_1_1_0
15:30:58	Processing ga-1 0_1_0_0_0_0
15:31:02	Processing mn-1 0_1_0_0_0_0
15:31:06	Processing ao-1 0_1_0_0_0_0
15:31:42	Processing bd-1 0_1_0_0_0_0
15:32:48	Processing bf-1 0_1_0_0_0_0
15:33:45	Processing bi-1 0_1_0_0_0_0
15:34:08	Processing bj-1 0_1_0_0_0_0
15:34:27	Processing bw-1 0_1_0_0_0_0
15:34:34	Processing cd-1 0_1_0_0_0_0
15:35:44	Processing cf-1 0_1_0_0_0_0
15:35:53	Processing cg-1 0_1_0_0_0_0
15:35:56	Processing ci-1 0_1_0_0_0_0
15:36:23	Processing cm-1 0_1_0_0_0_0
15:36:54	Processing dj-1 0_1_0_0_0_0
15:36:54	Processing er-1 0_1_0_0_0_0
15:37:02	Processing et-1 0_1_0_0_0_0
15:41:07	Processing fm-1 0_1_0_0_0_0
15:41:09	Processing gh-1 0_1_0_0_0_0
15:41:37	Processing gm-1 0_1_0_0_0_0
15:41:38	Processing gn-1 0_1_0_0_0_0
15:41:54	Processing gq-1 0_1_0_0_0_0
15:41:55	Processing gw-1 0_1_0_0_0_0
15:41:58	Processing hn-1 0_1_0_0_0_0
15:42:17	Processing ht-1 0_1_0_0_0_0
15:42:31	Processing ke-1 0_1_0_0_0_0
1

...

16:06:05	Processing zm-1 0_1_0_0_0_0
16:07:53	Processing zw-1 0_1_0_0_0_0
16:09:16	Processing gh-1 0_0_0_0_0_0
16:10:07	Processing mr-1 0_0_0_0_0_0
16:10:16	Processing ao-1 0_0_0_0_0_0
16:10:53	Processing bd-1 0_0_0_0_0_0
16:12:00	Processing bf-1 0_0_0_0_0_0
16:12:58	Processing bi-1 0_0_0_0_0_0
16:13:22	Processing bj-1 0_0_0_0_0_0
16:13:42	Processing bw-1 0_0_0_0_0_0
16:13:49	Processing cd-1 0_0_0_0_0_0
16:14:59	Processing cf-1 0_0_0_0_0_0
16:15:08	Processing cg-1 0_0_0_0_0_0
16:15:11	Processing ci-1 0_0_0_0_0_0
16:15:38	Processing cm-1 0_0_0_0_0_0
16:16:10	Processing dj-1 0_0_0_0_0_0
16:16:10	Processing er-1 0_0_0_0_0_0
16:16:18	Processing et-1 0_0_0_0_0_0
16:20:25	Processing fm-1 0_0_0_0_0_0
16:20:26	Processing ga-1 0_0_0_0_0_0
16:20:28	Processing gm-1 0_0_0_0_0_0
16:20:29	Processing gn-1 0_0_0_0_0_0
16:20:45	Processing gq-1 0_0_0_0_0_0
16:20:46	Processing gw-1 0_0_0_0_0_0
16:20:48	Processing hn-1 0_0_0_0_0_0
16:21:08	Processing ht-1 0_0_0_0_0_0
16:21:22	Processing ke-1 0_0_0_0_0_0
1

/home/gost/anaconda3/envs/Geog/lib/python3.7/site-packages/pandas/core/reshape/merge.py:946: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  'representation', UserWarning)


16:36:55	Processing sd-1 0_0_0_0_0_0
16:37:57	Processing sl-1 0_0_0_0_0_0
16:38:03	Processing sn-1 0_0_0_0_0_0
16:38:19	Processing so-1 0_0_0_0_0_0
16:38:33	Processing ss-1 0_0_0_0_0_0
16:39:07	Processing st-1 0_0_0_0_0_0
16:39:08	Processing sz-1 0_0_0_0_0_0
16:39:12	Processing td-1 0_0_0_0_0_0
16:39:46	Processing tg-1 0_0_0_0_0_0
16:39:58	Processing tl-1 0_0_0_0_0_0
16:40:01	Processing tz-1 0_0_0_0_0_0
16:42:33	Processing ug-1 0_0_0_0_0_0
16:44:19	Processing vu-1 0_0_0_0_0_0
16:44:21	Processing za-1 0_0_0_0_0_0


/home/gost/anaconda3/envs/Geog/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


16:45:03	Processing zm-1 0_0_0_0_0_0
16:46:53	Processing zw-1 0_0_0_0_0_0


gdal_translate -of GTiff -a_nodata 0 /media/gost/DATA1/GEP/Clusters/ao-1/rasterized_gep_0.01_1_1_0_1_1_0_2025_gt10.tif /media/gost/DATA1/GEP/Clusters/ao-1/rasterized_gep_0.01_1_1_0_1_1_0_2025_gt10_noData.tif
gdal_translate -of GTiff -a_nodata 0 /media/gost/DATA1/GEP/Clusters/bd-1/rasterized_gep_0.01_1_1_0_1_1_0_2025_gt10.tif /media/gost/DATA1/GEP/Clusters/bd-1/rasterized_gep_0.01_1_1_0_1_1_0_2025_gt10_noData.tif
gdal_translate -of GTiff -a_nodata 0 /media/gost/DATA1/GEP/Clusters/bf-1/rasterized_gep_0.01_1_1_0_1_1_0_2025_gt10.tif /media/gost/DATA1/GEP/Clusters/bf-1/rasterized_gep_0.01_1_1_0_1_1_0_2025_gt10_noData.tif
gdal_translate -of GTiff -a_nodata 0 /media/gost/DATA1/GEP/Clusters/bi-1/rasterized_gep_0.01_1_1_0_1_1_0_2025_gt10.tif /media/gost/DATA1/GEP/Clusters/bi-1/rasterized_gep_0.01_1_1_0_1_1_0_2025_gt10_noData.tif
gdal_translate -of GTiff -a_nodata 0 /media/gost/DATA1/GEP/Clusters/bj-1/rasterized_gep_0.01_1_1_0_1_1_0_2025_gt10.tif /media/gost/DATA1/GEP/Clusters/bj-1/rasterized_ge

find /media/gost/DATA1/GEP/Clusters -name "*0.01_1_1_0_1_1_0_2025_gt10_noData.tif" > tifFiles.txt
gdalbuildvrt combined_gep_rasterized_0.01_1_1_0_1_1_0_2025.vrt -input_file_list tifFiles.txt
